In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
import datetime

In [2]:
columns_for_bins = ['Elevation', 'Horizontal_Distance_To_Roadways']

**_Get data_**

In [3]:
dataset = pd.read_csv('train.csv')
dataset.drop("Id", axis=1, inplace=True)

**_Get test data_**

In [4]:
test_data = pd.read_csv('test.csv')
id_column = test_data["Id"].copy(deep=True)
test_data.drop("Id", axis=1, inplace=True)

**_Delete columns_**

In [5]:
# dataset.drop('Soil_Type9', axis=1, inplace=True)
# test_data.drop('Soil_Type9', axis=1, inplace=True)

**_Get training data_**

In [6]:
dataset

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,2607,243,23,258,7,660,170,251,214,1282,...,0,0,0,0,0,0,0,0,0,3
15116,2603,121,19,633,195,618,249,221,91,1325,...,0,0,0,0,0,0,0,0,0,3
15117,2492,134,25,365,117,335,250,220,83,1187,...,0,0,0,0,0,0,0,0,0,3
15118,2487,167,28,218,101,242,229,237,119,932,...,0,0,0,0,0,0,0,0,0,3


In [7]:
X = dataset.drop("Cover_Type", axis=1)
y = dataset["Cover_Type"]

In [8]:
X.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

**_Featire engineering average distances_**

In [9]:
X["Average_Elev_Vert_Hydr"] = X[['Elevation', 'Vertical_Distance_To_Hydrology']].mean(axis=1)
test_data["Average_Elev_Vert_Hydr"] = test_data[['Elevation', 'Vertical_Distance_To_Hydrology']].mean(axis=1)

X["Average_Hydro_Road"] = X[['Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways']].mean(axis=1)
test_data["Average_Hydro_Road"] = test_data[['Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways']].mean(axis=1)

X["Average_Hydro_Fire"] = X[['Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Fire_Points']].mean(axis=1)
test_data["Average_Hydro_Fire"] = test_data[['Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Fire_Points']].mean(axis=1)

X["Average_Road_Fire"] = X[['Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points']].mean(axis=1)
test_data["Average_Road_Fire"] = test_data[['Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points']].mean(axis=1)

**_Feature engineering average degrees_**

In [10]:
X["Average_Hillshade"] = X[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].mean(axis=1)
test_data["Average_Hillshade"] = test_data[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].mean(axis=1)

**_Feature engineering Substractions_**

In [11]:
def calc_diff(a, b):
    return abs(b - a)
X['Substr_Hydro_Road'] = X.apply(lambda x: calc_diff(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways']), axis=1)
test_data['Substr_Hydro_Road'] = test_data.apply(lambda x: calc_diff(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways']), axis=1)

X['Substr_Road_Fire'] = X.apply(lambda x: calc_diff(x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Substr_Road_Fire'] = test_data.apply(lambda x: calc_diff(x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

X['Substr_Aspect_Slope'] = X.apply(lambda x: calc_diff(x['Aspect'], x['Slope']), axis=1)
test_data['Substr_Aspect_Slope'] = test_data.apply(lambda x: calc_diff(x['Aspect'], x['Slope']), axis=1)

X['Substr_Elev_Vert'] = X.apply(lambda x: calc_diff(x['Elevation'], x['Vertical_Distance_To_Hydrology']), axis=1)
test_data['Substr_Elev_Vert'] = test_data.apply(lambda x: calc_diff(x['Elevation'], x['Vertical_Distance_To_Hydrology']), axis=1)

X['Substr_Hillshade_noon_3'] = X.apply(lambda x: calc_diff(x['Hillshade_Noon'], x['Hillshade_3pm']), axis=1)
test_data['Substr_Hillshade_noon_3'] = test_data.apply(lambda x: calc_diff(x['Hillshade_Noon'], x['Hillshade_3pm']), axis=1)

X['Substr_Elev_Road'] = X.apply(lambda x: calc_diff(x['Elevation'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Substr_Elev_Road'] = test_data.apply(lambda x: calc_diff(x['Elevation'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

def calc_diff_3(a, b, c):
    return abs(b - a - c)

X['Substr_Horizontal_Dist'] = X.apply(lambda x: calc_diff_3(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Substr_Horizontal_Dist'] = test_data.apply(lambda x: calc_diff_3(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

**_Feature Engineering Additions_**

In [12]:
def calc_add(a, b):
    return b + a
X['Add_Hydro_Road'] = X.apply(lambda x: calc_add(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways']), axis=1)
test_data['Add_Hydro_Road'] = test_data.apply(lambda x: calc_add(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways']), axis=1)

X['Add_Hydro_Fire'] = X.apply(lambda x: calc_add(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Add_Hydro_Fire'] = test_data.apply(lambda x: calc_add(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

X['Add_Road_Fire'] = X.apply(lambda x: calc_add(x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Add_Road_Fire'] = test_data.apply(lambda x: calc_add(x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

X['Add_elev_road'] = X.apply(lambda x: calc_add(x['Elevation'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Add_elev_road'] = test_data.apply(lambda x: calc_add(x['Elevation'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

def calc_add_3(a, b, c):
    return b + a + c

X['Add_Horizontal_Dist'] = X.apply(lambda x: calc_add_3(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)
test_data['Add_Horizontal_Dist'] = test_data.apply(lambda x: calc_add_3(x['Horizontal_Distance_To_Hydrology'], x['Horizontal_Distance_To_Roadways'], x['Horizontal_Distance_To_Fire_Points']), axis=1)

# X['Substr_Hillshade'] = X.apply(lambda x: calc_diff_3(x['Hillshade_9am'], x['Hillshade_Noon'], x['Hillshade_3pm']), axis=1)
# test_data['Substr_Hillshade'] = test_data.apply(lambda x: calc_diff_3(x['Hillshade_9am'], x['Hillshade_Noon'], x['Hillshade_3pm']), axis=1)

**_Eucledean distance_**

In [13]:
X['Distanse_to_Hydrolody'] = (X['Horizontal_Distance_To_Hydrology'] ** 2 + 
                                     X['Vertical_Distance_To_Hydrology'] ** 2) ** 0.5
test_data['Distanse_to_Hydrolody'] = (test_data['Horizontal_Distance_To_Hydrology'] ** 2 + 
                                     test_data['Vertical_Distance_To_Hydrology'] ** 2) ** 0.5

**_Additional info based on cover type_**

In [14]:
# ELU_CODE = {
#     1:2702,2:2703,3:2704,4:2705,5:2706,6:2717,7:3501,8:3502,9:4201,
#     10:4703,11:4704,12:4744,13:4758,14:5101,15:5151,16:6101,17:6102,
#     18:6731,19:7101,20:7102,21:7103,22:7201,23:7202,24:7700,25:7701,
#     26:7702,27:7709,28:7710,29:7745,30:7746,31:7755,32:7756,33:7757,
#     34:7790,35:8703,36:8707,37:8708,38:8771,39:8772,40:8776
# }

# def climatic_zone(input_df):
#     df = input_df.copy()
#     df['Climatic_Zone'] = input_df['Soil_Type'].apply(
#         lambda x: int(str(ELU_CODE[x])[0])
#     )
#     return df
# X = climatic_zone(X)
# test_data = climatic_zone(test_data)

**_Normalize data_**

In [15]:
columns_for_normalization = ['Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_To_Roadways', 'Add_Horizontal_Dist', 'Add_elev_road']
X[columns_for_normalization] = normalize(X[columns_for_normalization])
test_data[columns_for_normalization] = normalize(test_data[columns_for_normalization])

In [16]:
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Substr_Elev_Vert,Substr_Hillshade_noon_3,Substr_Elev_Road,Substr_Horizontal_Dist,Add_Hydro_Road,Add_Hydro_Fire,Add_Road_Fire,Add_elev_road,Add_Horizontal_Dist,Distanse_to_Hydrolody
0,2596,0.003931,0.000231,0.019887,0,0.039312,0.017035,0.017883,0.011408,0.483995,...,2596.0,84.0,3683.0,6027.0,768.0,6537.0,6789.0,0.684099,0.543194,258.000000
1,2590,0.004381,0.000156,0.016585,-6,0.030511,0.017211,0.018385,0.011813,0.486998,...,2596.0,84.0,3635.0,6047.0,602.0,6437.0,6615.0,0.689621,0.534094,212.084889
2,2804,0.009392,0.000608,0.018109,65,0.214874,0.015811,0.016082,0.009122,0.413599,...,2739.0,103.0,3317.0,3209.0,3448.0,6389.0,9301.0,0.603066,0.646582,275.769832
3,2785,0.010442,0.001213,0.016303,118,0.208170,0.016034,0.016034,0.008219,0.418429,...,2667.0,116.0,3426.0,3363.0,3332.0,6453.0,9301.0,0.606052,0.642903,269.235956
4,2595,0.003554,0.000158,0.012082,-1,0.030876,0.017373,0.018478,0.011845,0.487388,...,2596.0,84.0,3577.0,5934.0,544.0,6325.0,6563.0,0.692308,0.530346,153.003268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,2607,0.051450,0.004870,0.054626,7,0.139740,0.035994,0.053144,0.045310,0.271434,...,2600.0,37.0,1325.0,880.0,918.0,1540.0,1942.0,0.823407,0.465800,258.094944
15116,2603,0.024328,0.003820,0.127270,195,0.124254,0.050064,0.044434,0.018296,0.266403,...,2408.0,130.0,1278.0,1340.0,1251.0,1958.0,1943.0,0.789760,0.517928,662.354890
15117,2492,0.030834,0.005753,0.083988,117,0.077085,0.057526,0.050623,0.019099,0.273133,...,2375.0,137.0,1305.0,1217.0,700.0,1552.0,1522.0,0.846551,0.434205,383.293621
15118,2487,0.043478,0.007290,0.056756,101,0.063004,0.059620,0.061702,0.030981,0.242644,...,2386.0,118.0,1555.0,908.0,460.0,1150.0,1174.0,0.890130,0.362404,240.260276


**_Train model_**

In [17]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_jobs=-1, random_state=42)
model.fit(X, y)

ExtraTreesClassifier(n_jobs=-1, random_state=42)

**_Predict model_**

In [18]:
y_predicted = model.predict(test_data)
df_for_saving = pd.DataFrame(y_predicted, index=id_column, columns=["Cover_Type"])

**_Predict probability_**

In [19]:
predicted_probability = model.predict_proba(test_data)
df_probabilities = pd.DataFrame(predicted_probability, index=id_column)

In [ ]:
for i in range(df_probabilities.shape[0]):
    if df_probabilities.iloc[i].max() < 0.5:
        df_for_saving.iloc[i] = 2

**_Save dataset_**

In [ ]:
df_for_saving.to_csv('cover_type.csv', index_label="Id")

Combinations tried:
    - standard scaler

1 - Spruce/Fir

2 - Lodgepole Pine

3 - Ponderosa Pine

4 - Cottonwood/Willow

5 - Aspen

6 - Douglas-fir

7 - Krummholz

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
df2 = dataset.groupby(['Wilderness_Area1', 'Cover_Type'])['Wilderness_Area1'].count().unstack('Cover_Type')
df2[[1, 2, 3, 4, 5, 6, 7]].plot(kind='bar', stacked=True)

In [ ]:
df2 = dataset.groupby(['Wilderness_Area2', 'Cover_Type'])['Wilderness_Area2'].count().unstack('Cover_Type')
df2[[1, 2, 3, 4, 5, 6, 7]].plot(kind='bar', stacked=True)

In [ ]:
df2 = dataset.groupby(['Wilderness_Area3', 'Cover_Type'])['Wilderness_Area3'].count().unstack('Cover_Type')
df2[[1, 2, 3, 4, 5, 6, 7]].plot(kind='bar', stacked=True)

In [ ]:
df2 = dataset.groupby(['Wilderness_Area4', 'Cover_Type'])['Wilderness_Area4'].count().unstack('Cover_Type')
df2[[1, 2, 3, 4, 5, 6, 7]].plot(kind='bar', stacked=True)

**_Examining soil_**

In [ ]:
for i in range(1, 41):
    plt.rcParams["figure.figsize"] = (16,8)
    df2 = dataset.groupby([f'Soil_Type{i}', 'Cover_Type'])[f'Soil_Type{i}'].count().unstack('Cover_Type')
    plt.title(f'Soil_Type{i}')
    df2[[1, 2, 3, 4, 5, 6, 7]].plot(kind='bar', stacked=True)

In [ ]:
for i in range(1, 41):
    print(test_data[f'Soil_Type{i}'].value_counts())